In [21]:
import os 
import json
import pprint
import tweepy

# Load Twitter API credentials from external file

Create a json file named `twitter_credentials.json`

```
{
    "api_key": "...",
    "api_secret": "...",
    "access_token": "...",
    "access_secret": "...",
    "bearer_token": "..."
}
```

In [22]:
credentials_file = 'twitter_credentials.json'

with open(credentials_file) as credentials_data:
   api_data = json.load(credentials_data)

api_key = api_data['api_key']
api_secret = api_data['api_secret']
access_token = api_data['access_token']
access_secret = api_data['access_secret']
bearer_token = api_data['bearer_token']

In [23]:

client = tweepy.Client(
    consumer_key=api_key, 
    consumer_secret=api_secret, 
    access_token=access_token, 
    access_token_secret=access_secret, 
    bearer_token=bearer_token,
    wait_on_rate_limit=True,
    return_type=dict
)

In [31]:
accounts = ["mizzaro", "Miccighel_"]

data_folder = "datav2"
tweets_folder = f"{data_folder}/tweets"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

def get_followers(user, quantity):
    
    print(f"Processing user {user}")
    
    followers_of_user = []
    
    for item in tweepy.Cursor(
        api.followers, 
        screen_name=user, 
        skip_status=True, 
        include_user_entities=False
        ).items():
        
        json_data = item._json
        
        found_follower = {}
        found_follower["id"] = json_data["id"]
        found_follower["name"] = json_data["name"]
        found_follower["screen_name"] = json_data["screen_name"]
        found_follower["location"] = json_data["location"]
        found_follower["description"] = json_data["description"]
        found_follower["protected"] = json_data["protected"]
        found_follower["followers_count"] = json_data["followers_count"]
        found_follower["friends_count"] = json_data["friends_count"]
        
        if (found_follower not in followers_of_user) and\
            not found_follower["protected"] and\
            (found_follower["followers_count"] < 1000) and\
            (found_follower["friends_count"] < 1000):
            followers_of_user.append(found_follower)

        if len(followers_of_user) == quantity:
            break
            
    print(f"Found {len(followers_of_user)} followers for user {user}")

    return followers_of_user


def get_friends(user, quantity):
    
    print(f"Processing user {user}")
    
    friends_of_user = []
    
    for item in tweepy.Cursor(
        api.friends, 
        screen_name=user, 
        skip_status=True, 
        include_user_entities=False
        ).items():
        
        json_data = item._json
        
        found_friend = {}
        found_friend["id"] = json_data["id"]
        found_friend["name"] = json_data["name"]
        found_friend["screen_name"] = json_data["screen_name"]
        found_friend["location"] = json_data["location"]
        found_friend["description"] = json_data["description"]
        found_friend["protected"] = json_data["protected"]
        found_friend["friends_count"] = json_data["friends_count"]
        found_friend["followers_count"] = json_data["followers_count"]
        
        # append only when following/follower < 1000
        if (found_friend not in friends_of_user) and\
            not found_friend["protected"] and\
           (found_friend["friends_count"] < 1000) and\
            (found_friend["followers_count"] < 1000):
            friends_of_user.append(found_friend)

        # exit when friends == quantity
        if len(friends_of_user) == quantity:
            break
            
    print(f"Found {len(friends_of_user)} friends of user {user}")
    
    return friends_of_user

def get_tweets(user, quantity):

    tweets = []
    for item in tweepy.Cursor(
        api.user_timeline, 
        screen_name=user,
        tweet_mode="extended"
    ).items(quantity):
        raw_tweet = item._json
        tweet = {}
        tweet["id"]= raw_tweet["id"]
        tweet["created_at"] = raw_tweet["created_at"]
        tweet["full_text"] = raw_tweet["full_text"],
        tweet["favorite_count"]= raw_tweet["favorite_count"]
        tweets.append(tweet)
    return tweets

def get_followersv2(username, quantity):
    followers_of_user = []

    data_for_page = 10
    data_total = quantity
    next_token = None

    response = client.get_user(username=username)
    user = response['data']
    print(f"Processing user with id {user['id']} and username {user['username']}")
    while (next_token) or (len(followers_of_user) <= 0):
        if next_token:
            response = client.get_users_followers(
                id=user['id'], user_fields=['name', 'description', 'location'], 
                max_results=data_for_page, pagination_token=next_token
                )
        else:
            response = client.get_users_followers(
                id=user['id'], user_fields=['name', 'description', 'location'], 
                max_results=data_for_page
                )
        next_token = response['meta']['next_token'] if 'next_token' in response['meta'] else None   
        for data in response['data']:
            if(len(followers_of_user) < data_total):
                followers_of_user.append(data)
            
    print(f"Found {len(followers_of_user)} followers for user {username}")
    return followers_of_user

In [32]:
get_followersv2('mizzaro', 15)

Processing user mizzaro
Processing user with id 18932422 and username mizzaro


Rate limit exceeded. Sleeping for 897 seconds.


# First level followers

In [ ]:

first_level_followers = {}
for account in accounts:
    first_level_followers[account] = get_followers(account, 5)
    
serialize_json(data_folder, "first_level_followers.json", first_level_followers)

# First level friends

In [ ]:

first_level_friends = {}
for account in accounts:
    first_level_friends[account] = get_friends(account, 5)
    
serialize_json(data_folder, "first_level_friends.json", first_level_friends)

# Second level followers

In [ ]:
second_level_followers = {}
first_level_followers_data = read_json(f"{data_folder}/first_level_followers.json")

for followers_list in first_level_followers_data.values():
    for follower in followers_list:
        second_level_followers[follower['screen_name']] = get_followers(follower['screen_name'], 5)
    
serialize_json(data_folder, "second_level_followers.json", second_level_followers)

Data read from path: data/first_level_followers.json
Processing user OlegZendel
Found 5 followers for user OlegZendel
Processing user LyndaSaidLhadj
Found 5 followers for user LyndaSaidLhadj
Processing user acai2021
Found 5 followers for user acai2021
Processing user rodger_benham
Found 5 followers for user rodger_benham
Processing user ColoursRiot
Found 5 followers for user ColoursRiot
Processing user Il_Bale_
Found 1 followers for user Il_Bale_
Processing user Outsider289
Found 5 followers for user Outsider289
Processing user mizzaro
Found 5 followers for user mizzaro
Processing user EUROARTISTS
Found 5 followers for user EUROARTISTS
Processing user fra_bomba6
Found 4 followers for user fra_bomba6
Data serialized to path: data/second_level_followers.json


# Second level friends

In [ ]:
second_level_friends = {}
first_level_friends_data = read_json(f"{data_folder}/first_level_friends.json")

for friends_list in first_level_friends_data.values():
    for friends in friends_list:
        second_level_friends[friends['screen_name']] = get_friends(friends['screen_name'], 5)
    
serialize_json(data_folder, "second_level_friends.json", second_level_friends)

Data read from path: data/first_level_friends.json
Processing user MaxZancanaro
Found 5 friends of user MaxZancanaro
Processing user CIRES_AU
Found 5 friends of user CIRES_AU
Processing user mccart_richard
Found 5 friends of user mccart_richard
Processing user Miccighel_
Found 5 friends of user Miccighel_
Processing user rmit_cidda
Found 5 friends of user rmit_cidda
Processing user fra_bomba6
Found 5 friends of user fra_bomba6
Processing user Outsider289
Found 5 friends of user Outsider289
Processing user BiancolinoElisa
Found 5 friends of user BiancolinoElisa
Processing user ZuzanPink
Found 5 friends of user ZuzanPink
Processing user eglu81


Rate limit reached. Sleeping for: 866


Found 5 friends of user eglu81
Data serialized to path: data/second_level_friends.json


# Last 5 tweets of first and second level followers

In [ ]:
first_level_followers_data = read_json(f"{data_folder}/first_level_followers.json")
second_level_followers_data = read_json(f"{data_folder}/second_level_followers.json")

for followers_list in first_level_followers_data.values():
    for follower in followers_list:
        print(follower['screen_name'])
        if not os.path.exists(f"{tweets_folder}/{follower['screen_name']}.json"):
            tweets = get_tweets(follower['screen_name'], 5)
            serialize_json(tweets_folder, f"{follower['screen_name']}.json", tweets)


for followers_list in second_level_followers_data.values():
    for follower in followers_list:
        print(follower['screen_name'])
        if not os.path.exists(f"{tweets_folder}/{follower['screen_name']}.json"):
            tweets = get_tweets(follower['screen_name'], 5)
            serialize_json(tweets_folder, f"{follower['screen_name']}.json", tweets)

Data read from path: data/first_level_followers.json
Data read from path: data/second_level_followers.json
OlegZendel
LyndaSaidLhadj
acai2021
rodger_benham
ColoursRiot
Il_Bale_
Outsider289
mizzaro
EUROARTISTS
fra_bomba6
Marwah_k
ivrik
Dars2022_Sympo
ameertawfik
aleksbobic
gael_dias
Nthakur20
AthmaniAmine
iiia_unipd
OthmaniDr
TilburgU_DCA
srj_krishna
gabriellearuta
lesleywilton
sencerdeniz
NFT_Starr777
MaksudaBilkis
jparapar
ameertawfik
Jinxuan16807201
ciscaredda0111
AmbesiAntonio
BrillantVacant
LuvYahBoo24
Martha_Young
mrdanieltaylor1
Miccighel_
Sogno_Danzante
boreale01
nico_peresson
Giulia13568642
OlegZendel
LyndaSaidLhadj
acai2021
rodger_benham
ColoursRiot
modosmanagement
isabelleGILLOO1
JoziOpera
Tenor_Rguez27
MarigonaQerkezi
Miccighel_
Ilele99
cantaht
mary04655117
